de bai:
- dem so luong orders tuong ung voi moi kieu ship
- top 10 khach hang co orders nhieu nhat
- tong so luong khach hang
- khach hang co priority la high cao nhat
  giai bang dataframe, sql

In [2]:
from pyspark.sql import SparkSession

In [4]:
ss = SparkSession \
    .builder \
    .appName("thuc hanh dataframe 2") \
    .config("spark.sql.warehouse.dir", "D:\Python\2024\test_hadoop\hoc_spark\warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

In [11]:
orders_df = ss.read.csv("D:/Python/2024/test_hadoop/hoc_spark/demo.csv", header=True, inferSchema=True)

In [12]:
orders_df.show(5)

+--------------+-----------+-----------+----------+--------+
|Order_Priority|Customer_ID|  Ship_Mode|Order_Date|Order_ID|
+--------------+-----------+-----------+----------+--------+
| Not Specified|          2|Regular Air| 5/28/2012|   88525|
|          High|          3|Express Air|  7/7/2010|   88522|
| Not Specified|          3|Express Air| 7/27/2011|   88523|
| Not Specified|          3|Regular Air| 7/27/2011|   88523|
| Not Specified|          3|Express Air| 7/27/2011|   88523|
+--------------+-----------+-----------+----------+--------+
only showing top 5 rows



## tao bang tam

In [14]:
orders_df.createOrReplaceTempView("orders_tmp")

In [18]:
ss.sql("select * from orders_tmp").show(5)

+--------------+-----------+-----------+----------+--------+
|Order_Priority|Customer_ID|  Ship_Mode|Order_Date|Order_ID|
+--------------+-----------+-----------+----------+--------+
| Not Specified|          2|Regular Air| 5/28/2012|   88525|
|          High|          3|Express Air|  7/7/2010|   88522|
| Not Specified|          3|Express Air| 7/27/2011|   88523|
| Not Specified|          3|Regular Air| 7/27/2011|   88523|
| Not Specified|          3|Express Air| 7/27/2011|   88523|
+--------------+-----------+-----------+----------+--------+
only showing top 5 rows



## giai y 1: dem so luong order voi moi kieu ship mode

### gom cac kieu ship mode bang dataframe

In [21]:
orders_df.groupBy("Ship_Mode").count().show()

+--------------+-----+
|     Ship_Mode|count|
+--------------+-----+
|Delivery Truck| 1283|
|   Regular Air| 7036|
|   Express Air| 1107|
+--------------+-----+



### bang sql

In [22]:
ss.sql("select Ship_Mode, count(*) from orders_tmp group by Ship_Mode").show()

+--------------+--------+
|     Ship_Mode|count(1)|
+--------------+--------+
|Delivery Truck|    1283|
|   Regular Air|    7036|
|   Express Air|    1107|
+--------------+--------+



# giai y 2: top 10 khach orders nhieu nhat

## bang dataframe

In [31]:
khach_orders_nhieu_nhat = orders_df.groupBy("Customer_ID").count().sort("count", ascending=False).limit(10)

In [32]:
khach_orders_nhieu_nhat.show()

+-----------+-----+
|Customer_ID|count|
+-----------+-----+
|       1193|   27|
|        699|   26|
|       2107|   22|
|       2491|   22|
|       2882|   21|
|        308|   21|
|       3079|   20|
|        272|   19|
|       1999|   19|
|       1723|   18|
+-----------+-----+



## bang sql

In [35]:
a = ss.sql("select Customer_ID, count(*) as count from orders_tmp group by Customer_ID order by count desc limit 10").show()

+-----------+-----+
|Customer_ID|count|
+-----------+-----+
|       1193|   27|
|        699|   26|
|       2107|   22|
|       2491|   22|
|       2882|   21|
|        308|   21|
|       3079|   20|
|        272|   19|
|       1999|   19|
|       1723|   18|
+-----------+-----+



In [37]:
#print(a.type())

# y 3: tong so luong khach hang

## dataframe

In [39]:
tong_khach = orders_df.select("Customer_ID").distinct().count()

In [41]:
print(tong_khach)

2703


## sql

In [43]:
ss.sql("select count(distinct Customer_ID) as numbers_cust from orders_tmp").show()

+------------+
|numbers_cust|
+------------+
|        2703|
+------------+



# tim khach co priority high nhieu nhat

## dataframe

In [45]:
new_df = orders_df.where("Order_Priority = 'High'")

In [46]:
new_df.show(5)

+--------------+-----------+-----------+----------+--------+
|Order_Priority|Customer_ID|  Ship_Mode|Order_Date|Order_ID|
+--------------+-----------+-----------+----------+--------+
|          High|          3|Express Air|  7/7/2010|   88522|
|          High|          6|Regular Air| 5/26/2011|   90194|
|          High|          7|Regular Air|10/30/2012|   90198|
|          High|         12|Regular Air| 2/24/2012|   90196|
|          High|         16|Regular Air| 2/12/2010|   86836|
+--------------+-----------+-----------+----------+--------+
only showing top 5 rows



In [50]:
final = new_df.groupBy("Customer_ID").count().sort("count", ascending=False).limit(1)

In [51]:
final.show()

+-----------+-----+
|Customer_ID|count|
+-----------+-----+
|       1193|    9|
+-----------+-----+



## sql

In [55]:
ss.sql("select Customer_ID, count(*) as count from orders_tmp where Order_Priority = 'High' group by Customer_ID order by count desc limit 1").show()

+-----------+-----+
|Customer_ID|count|
+-----------+-----+
|       1193|    9|
+-----------+-----+

